In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import datetime
import random 
import copy 
import gc
import sys
sys.path.append("rl_method")
import env
import agent
import preprocess

import load_data
import utils

from torch.utils.data import Dataset, DataLoader
from tensorboardX import SummaryWriter
from sklearn.model_selection import train_test_split

In [3]:
model_path = '/home/willer/Desktop/Development/Python/MyRepo/npu-deeplearning-bci/model/PretrainNet_T1.pkl'
enet = preprocess.EncodeNet_T()
pnet = preprocess.PretrainNet_T()
pnet.load_state_dict(torch.load(model_path))

enet_dict = enet.state_dict()
for (name, param) in enet_dict.items():
    enet_dict[name] = copy.deepcopy(pnet.state_dict()[name])
enet.load_state_dict(enet_dict)
enet.eval()

n_classes = 2
ndata, nlabel = load_data.get_grazdata()

In [4]:
nlabel = nlabel.reshape(-1, 1)
train_loader, test_loader = load_data.boost_dataloader(ndata, nlabel, batch_size=512)

In [5]:
enet.to(torch.device('cuda'))
ndata = None
nlabel = None
with torch.no_grad():
    for input, label in train_loader:
        output = enet(input).cpu().numpy()
        label = label.cpu().numpy().reshape(-1)
        vec_label = np.eye(n_classes)[label]
        
        if str(type(ndata)) == "<class 'NoneType'>":
            ndata  = output
            nlabel = vec_label
        else:
            ndata  = np.concatenate([ndata, output], 0)
            nlabel = np.concatenate([nlabel, vec_label], 0)

    for input, label in test_loader:
        output = enet(input).cpu().numpy()
        label = label.cpu().numpy().reshape(-1)
        vec_label = np.eye(n_classes)[label]

        ndata  = np.concatenate([ndata, output], 0)
        nlabel = np.concatenate([nlabel, vec_label], 0)

print(ndata.shape, nlabel.shape)

(48416, 10, 128) (48416, 2)


In [6]:
np.save('rl_method/encode_data/encode_data_tem1.npy', ndata)
np.save('rl_method/encode_data/encode_label_tem1.npy', nlabel)

In [ ]:
ndata = np.load('rl_method/encode_data/encode_data_tem1.npy')
nlabel = np.load('rl_method/encode_data/encode_label_tem1.npy')

In [184]:

def get_reward_net(input_size, model_path):

    rnet = RewardNet(input_size)
    pnet = PretrainNet_T()
    pnet.load_state_dict(torch.load(model_path))

    rnet_dict = rnet.state_dict()
    for (name, param)  in rnet_dict.items():
        rnet_dict[name] = copy.deepcopy(pnet.state_dict()[name])
    rnet.load_state_dict(rnet_dict)
    rnet.eval()
    return rnet

class PretrainNet_T(nn.Module):

    def __init__(
        self,
        in_channel=3,
        sequence_lens=1000,
        time_lens=10,
        hidden_size=64,
        output_size=2,
        layer_size=1,
        bidirectional=True
    ):
        super(PretrainNet_T, self).__init__()

        if sequence_lens % time_lens != 0:
            raise ValueError("Invalid time lens")

        self.in_channel  = in_channel
        self.time_lens   = time_lens
        self.hidden_size = hidden_size
        self.layer_size  = layer_size
        self.window_size = sequence_lens // time_lens
        self.device      = torch.device('cuda')

        self.subconv    = SubConvNet(in_channel=in_channel, out_channel=4)
        self.input_size = self._adaptive_feature_size()

        self.lstm = nn.LSTM(self.input_size, hidden_size, layer_size, bidirectional=bidirectional)
        if bidirectional:
            self.layer_size *= 2

        self.fn1  = nn.Linear(hidden_size * self.layer_size, 128)
        self.fn2  = nn.Linear(128, output_size)

    def forward(self, x):

        batch_size = x.shape[0]

        x = x.chunk(self.time_lens, 2)
        x = torch.stack(x, 1)
        x = x.reshape(batch_size * self.time_lens, self.in_channel, self.window_size)

        x = self.subconv(x)
        x = x.view(batch_size, self.time_lens, self.input_size)
        x = x.permute(1, 0, 2)

        h_0 = torch.zeros(self.layer_size, batch_size, self.hidden_size).to(self.device)
        c_0 = torch.zeros(self.layer_size, batch_size, self.hidden_size).to(self.device)
        x, (h_final, c_final) = self.lstm(x, (h_0, c_0))
        # seq, batch, feature
        x = x.permute(1, 2, 0)

        x = F.avg_pool1d(x, self.time_lens)
        x = x.view(batch_size, -1)
        x = F.relu(self.fn1(x), inplace=True)
        x = F.softmax(self.fn2(x), dim=-1)
        return x

    def _adaptive_feature_size(self):
        x = torch.zeros(1, self.in_channel, self.window_size)
        return self.subconv(x).view(-1).shape[0]


In [189]:
import preprocess
def calculate_reward(vector_a, vector_b):
    return -np.sum(np.abs(vector_a - vector_b))

def cal_cosine_similarity(vector_a, vector_b):
    inner = np.dot(vector_a, vector_b.transpose())
    norm = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
    return inner / norm

class FeatureManager:
    def __init__(self, data):
        self.data = data
        self.size = data.shape[-1]
        self.index = list(range(data.shape[0]))

    def drop(self, action):
        self.index.remove(self.index[action])

    def state(self):
        new_state = self.data[self.index]
        new_size  = new_state.shape[0]
        new_index = list(range(new_size))

        avg_state = torch.mean(new_state, 0)
        ret_state = []
        for i in range(new_size):
            remaining = new_state[new_index[:i] + new_index[i+1:]]
            mean_state, var_state = self.mean_var_state(remaining)
            ret_state.append([new_state[i], mean_state, var_state])
        return avg_state, ret_state

    def mean_var_state(self, remaining_feature):
        shape = remaining_feature.shape[1]
        size  = remaining_feature.shape[0]
        mean_state = torch.mean(remaining_feature, 0)
        var_state  = torch.mean(torch.pow(remaining_feature - mean_state, 2), 0)
        return mean_state, var_state


class DropEnv:
    def __init__(self, tdata, vdata, tlabel, vlabel, drop_reward, reward_model_path):

        self.drop_reward = drop_reward
        self.random_stop = 0.8
        self.training = True
        
        self.tdata  = torch.from_numpy(tdata)
        self.vdata  = torch.from_numpy(vdata)
        self.tlabel = tlabel
        self.vlabel = vlabel
        
        self.tdata_size = self.tdata.shape[0]
        self.vdata_size = self.vdata.shape[0]
        self.channel_size = self.tdata.shape[1]
        
        self.reward_module = get_reward_net(tdata[0].shape[-1], reward_model_path)

    def step(self, action):
        self.manager.drop(action)
        avg_state, state = self.manager.state()
        cls_vector = self.reward_module(avg_state).numpy()
        self.new_sim = calculate_reward(cls_vector, self.current_label)
        if self.training:
            reward = self.new_sim - self.old_sim + self.drop_reward
        else:
            reward = self.new_sim - self.old_sim
        self.old_sim = self.new_sim

        done = False
        if reward < 0 and (self.training == True and random.random() < self.random_stop \
                           or self.training == False):
            done = True
        
        if self.training:
            return state, reward, done
        else:
            res = np.argmax(cls_vector)
            return state, float(res==self.current_num), done

    def train(self):
        self.training = True

    def eval(self):
        self.training = False

    def reset(self):
        if self.training:
            index = random.randint(0, self.tdata_size-1)
            single_data = self.tdata[index]
            self.current_label = self.tlabel[index]
        else:
            index = random.randint(0, self.vdata_size-1)
            single_data = self.vdata[index]
            self.current_label = self.vlabel[index]
            
        self.current_num = np.argmax(self.current_label)
        self.manager = FeatureManager(single_data)
            
        random_drop_num = 0
        
        if self.training:
            random_drop_num = random.randint(1, self.channel_size//4)
            random_drop_idx = random.sample(range(0, self.channel_size - random_drop_num - 1), random_drop_num)
            for idx in random_drop_idx:
                self.manager.drop(idx)

        init_avg_state, init_state = self.manager.state()
        init_cls_vector = self.reward_module(init_avg_state).numpy()
        init_cls_num = np.argmax(init_cls_vector)
        self.old_sim = calculate_reward(init_cls_vector, self.current_label)
        
        gc.collect()
        
        if self.training:
            return init_state, random_drop_num
        else:
            return init_state, float(init_cls_num==self.current_num), random_drop_num

In [197]:
ndata = ndata[:1000]
nlabel = nlabel[:1000]

In [ ]:
model_path = '/home/willer/Desktop/Development/Python/MyRepo/npu-deeplearning-bci/model/PretrainNet_T1.pkl'
agent = ADAgent(ndata, nlabel, model_path, train_epoch=8000)
agent.train()

In [77]:
def map_to_origin(action):
    res = [0 for i in range(10)]
    map_move = 0
    for act in action:
        res[act + map_move] += 1
        map_move += 1
    return res

In [79]:
map_to_origin([1,2,1,2])

[0, 1, 0, 2, 0, 1, 0, 0, 0, 0]

In [164]:
random.randint(0, 0)

0

In [173]:
a = np.array([0, 1])
b = np.array([0.1, 0.9])

In [188]:
np.argmax(b)

1